# Development Notebook

### Imports

In [64]:
# -- public imports

import os
import pandas as pd
from brat_parser import get_entities_relations_attributes_groups

In [2]:
# -- private import

In [3]:
# -- dev imports

%load_ext autoreload
%autoreload 2

## Importing the dataset

In [85]:
base_path = '../../data/UCL/dataset2'
path = os.path.join(base_path, 'ArgumentAnnotatedEssays-2.0')
text_file_dir = os.path.join(path, 'brat-project-final')

texts = []
annotated_texts = []
for file in os.listdir(text_file_dir):
    essay_num, file_extension = file.split('.')
    if file_extension == 'ann':
        with open(os.path.join(text_file_dir, file), 'r') as f:
            df_temp = pd.read_csv(f, delimiter='\t', header=None, names=['label_type', 'label', 'text'])
            df_temp[['label', 'label_comp1', 'label_comp2']] = df_temp.label.str.split(expand=True)
            df_temp['doc_id'] = essay_num
            annotated_texts.append(df_temp)
    elif file_extension == 'txt':
        with open(os.path.join(text_file_dir, file), 'r') as f:
            texts.append((essay_num, f.read()))
    else:
        continue

df_texts = pd.DataFrame.from_records(texts, columns={'doc_id', 'text'})
df_annotated = pd.concat(annotated_texts)
assert sorted(df_annotated.doc_id.unique()) == sorted(df_texts.doc_id)

In [95]:
df_texts.to_csv(
    os.path.join(base_path, 'df_texts.csv'), index=None
)

df_annotated.reset_index(drop=True).to_csv(
    os.path.join(base_path, 'df_full_annotated.csv'), index=None
)

In [103]:
ids_argument_segment = df_annotated.label_type.str.startswith('T')

df_arguments = df_annotated[ids_argument_segment]

df_arguments

,label_type,label,text,label_comp1,label_comp2,doc_id
0,T1,MajorClaim,alternative means of transportation and intern...,358,464,essay031
1,T2,MajorClaim,alternative forms of transport and internation...,2393,2565,essay031
2,T3,Claim,some people claim the convenience of automobile,2325,2372,essay031
4,T4,Claim,it is crucial to alter automobiles to the othe...,822,933,essay031
6,T5,Premise,an increase in the number of automobiles bring...,561,638,essay031
...,...,...,...,...,...,...
11417,T10,Premise,"Thanks to ads we learn about new products, we ...",1209,1305,essay028
11419,T11,Claim,"These days, not only many businesses, but also...",1307,1393,essay028
11421,T12,Premise,tourism makes up one-third of the Czech Republ...,1408,1466,essay028
11422,T13,Premise,"In order to promote the country's attractions,...",1468,1595,essay028


In [110]:
df_arguments = df_arguments.rename(columns={'label_comp1':'span_start', 'label_comp2':'span_end'}).astype(
    {'span_start': int, 'span_end':int}
)
df_arguments.dtypes

label_type    object
label         object
text          object
span_start     int64
span_end       int64
doc_id        object
dtype: object

In [133]:
records = []
df_arguments = df_arguments.sort_values(['doc_id', 'span_start', 'span_end'])
for (doc_id, text) in df_texts.sort_values('doc_id').itertuples(index=False):
    df_argument = df_arguments[df_arguments.doc_id == doc_id]
    prev_span = 0
    for (text_segment, span_start, span_end) in df_argument[['text', 'span_start', 'span_end']].itertuples(index=False):
        try:
            assert text_segment == text[span_start: span_end], f'{text_segment}\n{text[span_start: span_end]}'
        except Exception as e:
        # all the exception were manually checked. These are because of qutoe chars, this is a hot fix.!!!! TODO
            df_arguments['text'][(df_arguments.text == text_segment)] == text[span_start: span_end]
        
            
    
#(label_type, label, text, span_start, span_end, label) in df_arguments.sort_values(['span_start', 'span_end']).itertuples(index=False):
#    full_text = df.texts

In [132]:
with open(os.path.join(path, 'brat-project-final/essay120.ann'), 'r') as f:
    df_temp = pd.read_csv(f, delimiter='\t', header=None, names=['label_type', 'label', 'text'])
    print(df_temp)

   label_type                       label  \
0          T1          MajorClaim 323 384   
1          T2        MajorClaim 1700 1725   
2          T3             Claim 1605 1668   
3          A1           Stance T3 Against   
4          T4               Claim 422 514   
5          A2               Stance T4 For   
6          T5             Premise 525 624   
7          T6             Premise 626 675   
8          R1     attacks Arg1:T6 Arg2:T5   
9          R2     attacks Arg1:T5 Arg2:T4   
10         T7             Premise 690 742   
11         T8             Premise 759 869   
12         R3    supports Arg1:T8 Arg2:T4   
13         R4    supports Arg1:T7 Arg2:T4   
14         T9              Claim 881 1069   
15        T10           Premise 1081 1288   
16         A3               Stance T9 For   
17         R5   supports Arg1:T10 Arg2:T9   
18        T11             Claim 1290 1402   
19         A4              Stance T11 For   
20        T12           Premise 1463 1579   
21        